# Note to Reviewer: 
Several feature engineering steps were completed throughout my data preprocessing and EDA steps. 
I've completed troubleshooting for previous open questions:
- why do all zctas have at least one flood? I made the choice early on to only include zctas with floods, but went back and added all. ZCTAs with no floods could add value to model.
- I added a new dataset, National hydrography dataset (ie streamlines) and calculated total channel lenght within each zcta.
- I also added a count of flood by season (fall, winter, and spring) to use in the model 

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [2]:
data = pd.read_csv('data/02_data.csv', index_col=0)

data.isna().sum()

med_houseinc_12mon              0
household_public_assistance1    0
state_fips                      0
nflood_total                    0
flood_count_fall                0
flood_count_winter              0
flood_count_spring              0
nflood_2000s                    0
nflood_2010s                    0
flood_dur_hours_median          0
flood_dur_hours_min             0
flood_dur_hours_max             0
zcta                            0
channel_length_km               0
delta_floods                    0
dtype: int64

In [3]:
# keep zcta but remove from modeling data
data.index = data['zcta']

# drop columns not used in model
data_medianinc = data.drop(['zcta', 'state_fips', 'household_public_assistance1'], axis=1)

data_medianinc


,med_houseinc_12mon,nflood_total,flood_count_fall,flood_count_winter,flood_count_spring,nflood_2000s,nflood_2010s,flood_dur_hours_median,flood_dur_hours_min,flood_dur_hours_max,channel_length_km,delta_floods
zcta,,,,,,,,,,,,
35004,76683.0,1,0,0,0,1.0,0.0,1.967213,1.967213,1.967213,64.301,-1.000000
35005,55017.0,3,1,0,0,2.0,1.0,3.442623,1.967213,3.934426,94.617,-0.333333
35006,63521.0,2,0,1,0,0.0,1.0,3.196721,1.967213,4.426230,306.497,1.000000
35007,81351.0,6,1,0,1,1.0,4.0,2.459016,0.983607,8.852459,108.522,0.600000
35010,46730.0,5,0,0,1,2.0,2.0,1.721311,0.983607,8.360656,649.526,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
99919,64135.0,0,0,0,0,0.0,0.0,0.000000,0.000000,0.000000,0.000,0.000000
99921,59219.0,0,0,0,0,0.0,0.0,0.000000,0.000000,0.000000,0.000,0.000000
99922,53125.0,0,0,0,0,0.0,0.0,0.000000,0.000000,0.000000,0.000,0.000000


### Scale Data
I'll scale data later for each individual model. Planning on starting with RFR which doesn't need scaling (if fact, prefer not to for optimal interpretability). 

### Categorical data have already been removed in step 2 EDA. 

### split test/train
Split data using 80/20 train/test split for now. Planning to complete cross-validation check after first model built to check variances. 

In [4]:
# split test/train data
X = data_medianinc.drop('med_houseinc_12mon', axis=1)
y = data_medianinc['med_houseinc_12mon']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=777)



### Ready to begin modeling